In [ ]:
!pip uninstall transformers

Found existing installation: transformers 4.24.0
Uninstalling transformers-4.24.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.24.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.24.0


In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.24.0-py3-none-any.whl (5.5 MB)


In [ ]:
!pip install pyserini
!pip install jsonlines
!apt-get install libomp-dev
!pip install faiss

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/wikipedia_20220620_cleaned.zip

In [ ]:
!unzip /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/e2eqa-train+public_test-v1

Archive:  /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/e2eqa-train+public_test-v1.zip
   creating: e2eqa-train+public_test-v1/
  inflating: e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json  
  inflating: e2eqa-train+public_test-v1/zac2022_testa_only_question.json  
  inflating: e2eqa-train+public_test-v1/zac2022_train_merged_final.json  


In [ ]:
!unzip /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/lookup_db.zip 

Archive:  /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/lookup_db.zip
   creating: lookup_db/
  inflating: lookup_db/_6.fdt        
  inflating: lookup_db/_6.nvm        
  inflating: lookup_db/_6_Lucene90_0.dvm  
  inflating: lookup_db/_6.tvd        
  inflating: lookup_db/_6_Lucene90_0.tim  
  inflating: lookup_db/_6.si         
  inflating: lookup_db/segments_v    
 extracting: lookup_db/write.lock    
  inflating: lookup_db/_6.tvx        
  inflating: lookup_db/_6.fdx        
  inflating: lookup_db/_6_Lucene90_0.doc  
  inflating: lookup_db/_6_Lucene90_0.tip  
  inflating: lookup_db/_6_Lucene90_0.dvd  
  inflating: lookup_db/_6_Lucene90_0.pos  
  inflating: lookup_db/_6_Lucene90_0.tmd  
  inflating: lookup_db/_6.fnm        
  inflating: lookup_db/_6.fdm        
  inflating: lookup_db/_6.tvm        
  inflating: lookup_db/_6.nvd        


In [ ]:
import json
import os
import joblib
from tqdm.auto import tqdm
from transformers import pipeline
from pyserini.search.lucene import LuceneSearcher

In [ ]:
# question classification model 
quest_type_map = {'entity':0, 'date':1, 'number':2}
quest_type_map_reversed = {v:k for k,v in quest_type_map.items()}
quest_clf = joblib.load('/content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/models/question_classifier.pkl')

In [ ]:
VERSION = 'baseline'
OUT_DIR = f'/content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/valid_predictions/'
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# model_checkpoint = "nguyenvulebinh/vi-mrc-large"
model_checkpoint = "ancs21/xlm-roberta-large-vi-qa"

nlp = pipeline('question-answering', model=model_checkpoint,
                   tokenizer=model_checkpoint, device='cuda:0')

#{'score': 0.5782045125961304, 'start': 45, 'end': 68, 'answer': 'xử lý ngôn ngữ tự nhiên'}

Downloading:   0%|          | 0.00/789 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/179 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
searcher = LuceneSearcher('lookup_db')
searcher.set_language('vn')

## Demo 1 example

In [ ]:
question = 'nhạc sĩ nào là tác giả của bài hát nổi tiếng dáng đứng bến tre'

def retrieve_doc(question):
    hits = searcher.search(question)

    # for i in range(len(hits)):
    #     print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

    doc = searcher.doc(hits[0].docid)
    json_doc = json.loads(doc.raw())
    return json_doc['contents']

In [ ]:
retrieved_doc = retrieve_doc(question)

In [ ]:
import re

def get_answer(question, retrieved_doc):
    QA_input = {
    'question': question,
    'context': retrieved_doc
    }
    res = nlp(QA_input)['answer']
    res = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,\\]", "", res).strip()
    return res

In [ ]:
get_answer(question, retrieved_doc)

'Nguyễn Văn Tý'

In [ ]:
quest_clf.predict([question])[0]

0

# Predict on the whole train set (Full anno only)

In [ ]:
with open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json', 'r') as f:
    raw_data = json.load(f)['data']

In [ ]:
def is_date(answer_lower):
    checks = ['năm', 'tháng', 'ngày', 'thế kỷ', 'thời', 'thiên niên kỷ']
    for c in checks:
        if c in answer_lower:
            return True
    return False
    
def get_category(answer):
    answer_lower = answer.lower()
    if answer_lower.startswith('wiki'):
        return 'entity'
    if is_date(answer_lower):
        return 'date'
    if answer_lower.isdigit():
        return 'number'
    return None

In [ ]:
filtered = []
for doc in raw_data:
    if doc['category'] != 'FULL_ANNOTATION':
        continue
    # doc['question_type'] = get_category(doc['answer'])
    filtered.append(doc)

In [ ]:
len(filtered)

4989

In [ ]:
results = []
for doc in tqdm(filtered):
    doc['predicted_question_type'] = quest_type_map_reversed[
                                        quest_clf.predict([doc['question']])[0]
                                    ]
    doc['retrieved'] = retrieve_doc(doc['question']) 

    answer = get_answer(doc['question'], doc['retrieved'])

    if doc['predicted_question_type'] == 'entity':
        answer = 'wiki/' + re.sub('\\s+', '_', answer)

    doc['predicted_answer'] = answer
    results.append(doc)
    # break

  0%|          | 0/4989 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [32]:
with open(OUT_DIR + '/' + f'{VERSION}.json', 'w') as f:
    json.dump(results, f, ensure_ascii=False)